## Image tag parsing and annotation preview
Version 2021-01-04

### Load libraries

In [ ]:
# Library for data visualisation
# pip install opencv-python
from cv2 import cv2

# Built-in package for json parsing
import json

# Library for handling data paths
# pip install pathlib
import pathlib


## Select data location

Set *root_dir* to point to "Supervisely" project folder

In [ ]:
#Set path to root dir (wrist folder)
root_dir = pathlib.Path('Path_to_Supervisely_project_folder')

# Path to annotation dir
ann_dir = pathlib.Path(root_dir, 'ann')

# Path to image dir
img_dir = pathlib.Path(root_dir, 'img')

# Lists
list_files = lambda start_dir: [str(item) for item in start_dir.iterdir()]
ann_list = list_files(ann_dir)

print('Number of annotations: {}'.format(len(ann_list)))

## Functions used to parse and preview data 

In [ ]:
# Function that is parsing json file and printing all usefull data
def parse_json(path_to_annotation):
    #Load data
    with open(path_to_annotation) as json_file:  
        # Load json
        data = json.load(json_file)
        
        # Print tags:
        print('Tags:')
        for tag in data['tags']:
            print(tag)
            
        print('*******************')
        print('Objects:')
        #Print objects:
        for object_dict in data['objects']:
            print('Object type: {}\nObject points: {}'.format(object_dict['classTitle'], 
                                                                   object_dict['points']['exterior']))
            print('*******************')    
            

def display_annotations(path_to_annotation, image_dir, objectname, scale = 1):
    #Load data
    with open(path_to_annotation) as json_file:
        
        # Get name of the file
        name = pathlib.PurePath(path_to_annotation).name.split('.')[0]
        image = image_dir / (str(name) + '.png')

        # Load image
        original_image = cv2.imread(str(image))
        original_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        annotated_image = original_image.copy()
        
        # Load json
        data = json.load(json_file)
        
        # Draw fractures
        for object_dict in data['objects']:
            # Check if object is fracture
            if object_dict['classTitle'] == objectname:
               
                # Get points and convert them to int for display purposes
                top_left_point, bottom_right_point = object_dict['points']['exterior']
                top_left_point = list(map(int,top_left_point))
                bottom_right_point = list(map(int, bottom_right_point))
                
                # Draw points
                cv2.rectangle(annotated_image, (top_left_point[0], top_left_point[1]), 
                              (bottom_right_point[0], bottom_right_point[1]), (0, 0, 255), 5)
        
        # Scale images for display purposes
        original_image = cv2.resize(original_image, (0,0), fx=scale, fy=scale) 
        annotated_image = cv2.resize(annotated_image, (0,0), fx=scale, fy=scale) 
        
        # Display images
        cv2.imshow('Original', original_image)
        cv2.imshow('Annotated', annotated_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()

## Main function

Select index name of the image/object that you want to preview.

In [ ]:
# Select index of the image to preview
index = 0
objectname = 'fracture'

# To close images press any key
parse_json(ann_list[index])
display_annotations(ann_list[index], img_dir, objectname, scale = 0.75)